In [1]:
from mmdet3d.apis import init_model, inference_detector, show_result_meshlab, inference_multi_modality_detector

config_file = '/home/wistful/work/mmdetection3d/configs/mvxnet/dv_mvx-fpn_second_secfpn_adamw_2x8_80e_kitti-3d-3class.py'
checkpoint_file = '/home/wistful/ResultDir/my_pth/mxvnet/dv_mvx-fpn_second_secfpn_adamw_2x8_80e_kitti-3d-3class_20210831_060805-83442923.pth'

In [2]:
model = init_model(config_file, checkpoint_file, 'cuda:1')

/home/wistful/work/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:85: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  'dir_offset and dir_limit_offset will be depressed and be '


load checkpoint from local path: /home/wistful/ResultDir/my_pth/mxvnet/dv_mvx-fpn_second_secfpn_adamw_2x8_80e_kitti-3d-3class_20210831_060805-83442923.pth


In [3]:
from torchvision.transforms import transforms  # 取数据
from mmdet3d.datasets import build_dataset
from mmcv import Config

cfg = Config.fromfile(config_file)
# 读取数据集
datasets = [build_dataset(cfg.data.train)]

In [4]:
one_data = datasets[0][0]
print(one_data.keys())
one_data

dict_keys(['img_metas', 'points', 'img', 'gt_bboxes_3d', 'gt_labels_3d'])


{'img_metas': DataContainer({'filename': '/home/wistful/work/mmdetection3d/data/kitti/training/image_2/000000.png', 'ori_shape': (370, 1224, 3), 'img_shape': (264, 873, 3), 'lidar2img': array([[ 6.02943726e+02, -7.07913330e+02, -1.22748432e+01,
         -1.70942719e+02],
        [ 1.76777252e+02,  8.80879879e+00, -7.07936157e+02,
         -1.02568634e+02],
        [ 9.99984801e-01, -1.52826728e-03, -5.29071223e-03,
         -3.27567995e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]], dtype=float32), 'pad_shape': (288, 896, 3), 'scale_factor': array([0.7132353, 0.7135135, 0.7132353, 0.7135135], dtype=float32), 'flip': True, 'pcd_horizontal_flip': True, 'pcd_vertical_flip': False, 'box_mode_3d': <Box3DMode.LIDAR: 0>, 'box_type_3d': <class 'mmdet3d.core.bbox.structures.lidar_box3d.LiDARInstance3DBoxes'>, 'img_norm_cfg': {'mean': array([103.53 , 116.28 , 123.675], dtype=float32), 'std': array([1., 1., 1.], dtype=float32), 'to_rgb': False}, 'pcd_

In [5]:
extract_img_feat = model.extract_img_feat
# 获取图像特征，此处获取的是图像经过骨干和neck之后的数据，为5个通道数为256的特征
img_feats = extract_img_feat((one_data.get('img').data).unsqueeze(dim=0).cuda(), [one_data.get('img_metas').data])
for i in img_feats:
    print(i.shape)

torch.Size([1, 256, 72, 224])
torch.Size([1, 256, 36, 112])
torch.Size([1, 256, 18, 56])
torch.Size([1, 256, 9, 28])
torch.Size([1, 256, 5, 14])


In [6]:
extract_pts_feat = model.extract_pts_feat
# 获取图像特征，此处同上面各个字段的类型需要去代码里看定义
img_feat_list = list(img_feats)
pts_feats = extract_pts_feat([one_data.get('points').data.cuda()], img_feat_list, [one_data.get('img_metas').data])

/home/wistful/work/mmdetection3d/mmdet3d/models/fusion_layers/coord_transform.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if 'pcd_rotation' in img_meta else torch.eye(


In [7]:
# 此处的head为Anchor3DHead，返回值有三个： cls_score, bbox_pred, dir_cls_preds
# 其中，clas_score 通道数为  num_classes * num_anchors, num_classes在配置文件中
# bbox_pred 通道数为 num_anchors * box_code_size
# dir_cls_preds 通道数为 num_anchors * 2
# 得到head的输出后，还需要运行一下解码模块，才能得到最终的bbox和分类情况
pts_bbox_head = model.pts_bbox_head
pts_out = pts_bbox_head(pts_feats)  # tuple[list[torch.Tensor]]
cls_score, bbox_pred, dir_cls_preds = pts_out
print("cls_score:",cls_score[0].shape)
print("bbox_pred:",bbox_pred[0].shape)
print("dir_cls_preds:",dir_cls_preds[0].shape)

cls_score: torch.Size([1, 18, 200, 176])
bbox_pred: torch.Size([1, 42, 200, 176])
dir_cls_preds: torch.Size([1, 12, 200, 176])


In [8]:
# 将head得到的输出编码为bboxer
bboxes = model.pts_bbox_head.get_bboxes(cls_score,bbox_pred,dir_cls_preds,[one_data.get('img_metas').data])
print(type(bboxes[0][0])) # 是在LiDAR坐标系下
bboxes_data = bboxes[0][0] # 得到了4个预测框
bboxes_data

<class 'mmdet3d.core.bbox.structures.lidar_box3d.LiDARInstance3DBoxes'>


LiDARInstance3DBoxes(
    tensor([[  8.5166,  -1.4354,  -1.6564,   1.2415,   0.5879,   1.9521,   1.2375],
        [ 14.0160, -21.8561,  -1.8352,   1.7213,   0.5980,   1.6781,   4.0820],
        [  8.7474,  -1.3463,  -1.5435,   4.0260,   1.8987,   1.7594,  -0.3487]],
       device='cuda:1'))

In [9]:
# from mmdet3d.core import Box3DMode
#  # 将LiDAR BBOX转换到深度坐标系
# gt_bboxes_cam = bboxes_data.convert_to(dst=Box3DMode.DEPTH, rt_mat=None)
# # gt_bboxes_cam = gt_bboxes_cam.tensor
# gt_bboxes_cam

In [10]:
from PIL import Image
import cv2
from torchvision import transforms

trans = transforms.Compose([transforms.ToTensor()])

img_metas = one_data.get('img_metas').data
img_file_path = img_metas['filename']

img = cv2.imread(img_file_path)

front_mat = one_data.get('img_metas').data.get('lidar2img')
# front_mat
from mmdet3d.core import show_multi_modality_result
# image2CV = img.permute(1,2,0) # 交换维度，转换为openCV支持的格式：[H, W, C]


gt_boxes = one_data.get('gt_bboxes_3d').data
print(gt_boxes)
print(bboxes_data)
# gt_bboxes_cam
bboxes_data = bboxes_data.to('cpu')
show_multi_modality_result(img=img,
                           box_mode='lidar',
                           gt_bboxes=gt_boxes,
                           img_metas=img_metas,
                           pred_bboxes=bboxes_data,
                           proj_mat=front_mat,
                           out_dir="/home/wistful/work/mmdetection3d/visual_img/",
                           filename="test",
                           show=False)

LiDARInstance3DBoxes(
    tensor([[ 8.5271, -1.3907, -1.7189,  1.2102,  0.4841,  1.9061,  1.2355]]))
LiDARInstance3DBoxes(
    tensor([[  8.5166,  -1.4354,  -1.6564,   1.2415,   0.5879,   1.9521,   1.2375],
        [ 14.0160, -21.8561,  -1.8352,   1.7213,   0.5980,   1.6781,   4.0820],
        [  8.7474,  -1.3463,  -1.5435,   4.0260,   1.8987,   1.7594,  -0.3487]],
       device='cuda:1'))


In [11]:
import torch
all_boxes = torch.cat((bboxes_data.tensor,gt_boxes.tensor))
all_boxes

tensor([[  8.5166,  -1.4354,  -1.6564,   1.2415,   0.5879,   1.9521,   1.2375],
        [ 14.0160, -21.8561,  -1.8352,   1.7213,   0.5980,   1.6781,   4.0820],
        [  8.7474,  -1.3463,  -1.5435,   4.0260,   1.8987,   1.7594,  -0.3487],
        [  8.5271,  -1.3907,  -1.7189,   1.2102,   0.4841,   1.9061,   1.2355]])

In [12]:
point_file = "/home/wistful/work/mmdetection3d/demo/data/kitti/kitti_000008.bin"
img_file = '/home/wistful/work/mmdetection3d/demo/data/kitti/kitti_000008.png'
ann_file = '/home/wistful/work/mmdetection3d/demo/data/kitti/kitti_000008_infos.pkl'
out_dir = '/home/wistful/work/mmdetection3d/visual_img'
result, data = inference_multi_modality_detector(model=model, pcd=point_file,image=img_file, ann_file=ann_file)

In [13]:
from mmdet3d.apis import show_result_meshlab
"""
assert task in ['det', 'multi_modality-det', 'seg', 'mono-det'], \
        f'unsupported visualization task {task}'
"""
show_result_meshlab(data,result,out_dir,show=False,score_thr=0.3,task='multi_modality-det')

('/home/wistful/work/mmdetection3d/visual_img', 'kitti_000008')